In [1]:
# NOTE: Import libraries

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [11]:
# NOTE: Device configuration and hyperparameters

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

num_layers = 2
input_size = 28
hidden_size = 128
num_classes = 10
batch_size = 100
num_epochs = 2
learning_rate = 0.001
sequence_size = 28

print(device)

cpu


In [5]:
# NOTE: MNIST dataset

train_dataset = torchvision.datasets.MNIST(
    root='../data',
    train=True,
    transform=transforms.ToTensor(),
    download=True
)

test_dataset = torchvision.datasets.MNIST(
    root='../data',
    train=False,
    transform=transforms.ToTensor()
)

# NOTE: Data loader

train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=False
)

In [6]:
# NOTE: Recurrent neural network (many-to-one)

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])

        return out

In [12]:
# NOTE: Define model and loss function

model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, sequence_size, input_size).to(device)
        labels = labels.to(device)

        output = model(images)
        loss = criterion(output, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print (f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

with torch.no_grad():
    n_correct = 0
    n_samples = 0

    for images, labels in test_loader:
        images = images.reshape(-1, sequence_size, input_size).to(device)
        labels = labels.to(device)

        outputs = model(images)

        _, predicted = torch.max(outputs.data, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the model on the {n_samples} test images: {acc} %')

Epoch [1/2], Loss: 0.5761
Epoch [1/2], Loss: 0.2971
Epoch [1/2], Loss: 0.2580
Epoch [1/2], Loss: 0.2021
Epoch [1/2], Loss: 0.1319
Epoch [1/2], Loss: 0.1531
Epoch [2/2], Loss: 0.0837
Epoch [2/2], Loss: 0.1569
Epoch [2/2], Loss: 0.0983
Epoch [2/2], Loss: 0.0768
Epoch [2/2], Loss: 0.1706
Epoch [2/2], Loss: 0.0531
Accuracy of the model on the 10000 test images: 97.28 %


In [15]:
for images, labels in test_loader:
    images = images.reshape(-1, sequence_size, input_size)
    print(images.shape, labels.shape)

torch.Size([100, 28, 28]) torch.Size([100])
torch.Size([100, 28, 28]) torch.Size([100])
torch.Size([100, 28, 28]) torch.Size([100])
torch.Size([100, 28, 28]) torch.Size([100])
torch.Size([100, 28, 28]) torch.Size([100])
torch.Size([100, 28, 28]) torch.Size([100])
torch.Size([100, 28, 28]) torch.Size([100])
torch.Size([100, 28, 28]) torch.Size([100])
torch.Size([100, 28, 28]) torch.Size([100])
torch.Size([100, 28, 28]) torch.Size([100])
torch.Size([100, 28, 28]) torch.Size([100])
torch.Size([100, 28, 28]) torch.Size([100])
torch.Size([100, 28, 28]) torch.Size([100])
torch.Size([100, 28, 28]) torch.Size([100])
torch.Size([100, 28, 28]) torch.Size([100])
torch.Size([100, 28, 28]) torch.Size([100])
torch.Size([100, 28, 28]) torch.Size([100])
torch.Size([100, 28, 28]) torch.Size([100])
torch.Size([100, 28, 28]) torch.Size([100])
torch.Size([100, 28, 28]) torch.Size([100])
torch.Size([100, 28, 28]) torch.Size([100])
torch.Size([100, 28, 28]) torch.Size([100])
torch.Size([100, 28, 28]) torch.